In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
raw = pd.read_csv("data_sets/Books_Raw.csv", na_values='None')

In [3]:
start_time = time.time()
reviews = pd.read_csv("data_sets/Reviews_Raw.csv", na_values='None')
end_time = time.time()
print("Import time:", end_time - start_time, "seconds")

Import time: 32.892329931259155 seconds


In [4]:
# Change the values of missing
missing_val = raw["authors"][212400]
reviews = reviews.replace(missing_val, None)
raw = raw.replace(missing_val, None)

In [5]:
clean_books = raw[["Title", "publisher", "publishedDate"]].copy()

## Clean the book file

In [6]:
def process_string(in_str):
    if in_str is None:
        return []
    else:
        temp = in_str[1:-1].split(", ")
        temp = [mystr[1:-1] for mystr in temp]
        return temp

clean_books["authors"] = raw["authors"].apply(process_string)

In [7]:
type(raw["authors"].iloc[-2])

str

In [8]:
def my_count(str):
    if str == None:
        return None
    else:
        return str.count("'")
        
raw["categories_count"] = raw["categories"].apply(my_count)

In [9]:
raw[raw["categories_count"] > 2.0]

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount,categories_count


In [10]:
# No book has more than 1 category, so let's safely remove the quotes and brackets like we did for authors

# Remove the outer square brackets. Remove all quote marks. (Note some fields have multiple sets of quotes, if there are multiple authors)
def remove_outer_braces(str):
    if str is None:
        return None
    else:
        return str[2:-2]

clean_books["categories"] = raw["categories"].apply(remove_outer_braces)

In [11]:
clean_books

,Title,publisher,publishedDate,authors,categories
0,Its Only Art If Its Well Hung!,None,1996,[Julie Strain],Comics & Graphic Novels
1,Dr. Seuss: American Icon,A&C Black,2005-01-01,[Philip Nel],Biography & Autobiography
2,Wonderful Worship in Smaller Churches,None,2000,[David R. Ray],Religion
3,Whispers of the Wicked Saints,iUniverse,2005-02,[Veronica Haddon],Fiction
4,"Nation Dance: Religion, Identity and Cultural ...",None,2003-03-01,[Edward Long],None
...,...,...,...,...,...
212399,The Orphan Of Ellis Island (Time Travel Advent...,Scholastic Paperbacks,2000-06-01,[Elvira Woodruff],Juvenile Fiction
212400,Red Boots for Christmas,None,1995,[],Juvenile Fiction
212401,Mamaw,None,2018-01-17,[Wild Wild Cabbage],None
212402,The Autograph Man,Vintage,2003-08-12,[Zadie Smith],Fiction


In [12]:
# Notice one book does not have a title
print(clean_books[clean_books["Title"].isnull()])

# Let's remove it
clean_books.dropna(subset="Title", inplace=True)

     Title publisher publishedDate            authors categories
1066  None      None    2015-12-15  [Maharshi Ramana]       None


## Clean the reviews file

In [13]:
# Drop all reviews that don't have a title, as they can't be match to a book
reviews.dropna(subset="Title", inplace=True)
reviews.count()

Id                    2999792
Title                 2999792
Price                  481164
User_id               2438018
profileName           2437900
review/helpfulness    2999792
review/score          2999792
review/time           2999792
review/summary        2999385
review/text           2999784
dtype: int64

## Add review data to the raw data

In [15]:
number_of_reviews_per_title = reviews["Title"].value_counts()
number_of_reviews_per_title

Title
The Hobbit                                                                                                         22023
Pride and Prejudice                                                                                                20371
Atlas Shrugged                                                                                                     12513
Wuthering Heights                                                                                                  10780
The Giver                                                                                                           7644
                                                                                                                   ...  
Illuminatus! Part I: The Eye in the Pyramid (The eye in the Pyramid, The Golden Apple,Leviathan, Parts 1, 2, 3)        1
Modern Chess Openings                                                                                                  1
Child Abuse and Neglect: C

In [16]:
average_score_per_title = reviews.groupby("Title")["review/score"].mean().sort_index()
std_score_per_title = reviews.groupby("Title")["review/score"].std().sort_index()
std_score_per_title

Title
" Film technique, " and, " Film acting "                                                                                                   0.707107
" We'll Always Have Paris": The Definitive Guide to Great Lines from the Movies                                                            0.000000
"... And Poetry is Born ..." Russian Classical Poetry                                                                                           NaN
"A Titanic hero" Thomas Andrews, shipbuilder                                                                                               0.353553
"A Truthful Impression of the Country": British and American Travel Writing in China, 1880-1949                                                 NaN
                                                                                                                                             ...   
with an everlasting love                                                                                  

In [17]:
# Notice some of these titles are WILD
print(any(i == '" Film technique, " and, " Film acting "' for i in reviews["Title"]))
print(any(i == 'www.whitbread.org/book' for i in reviews["Title"]))

True
True


In [18]:
# The titles with a std of None only have 1 review. Probably we will want to change this to 0, but there is no need to do that know.
# It is unclear to LS whether this std is a biased or an un-biased estimater. (recall an 1/(m-1) for un-biased variance estimators)
print(number_of_reviews_per_title['"... And Poetry is Born ..." Russian Classical Poetry'])

1


In [19]:
clean_alphabetically = clean_books.sort_values(by="Title")

clean_alphabetically["ratings_average"] = average_score_per_title.tolist()
clean_alphabetically["ratings_std"]  = std_score_per_title.tolist()
clean_alphabetically["ratings_count"] = number_of_reviews_per_title.sort_index().tolist()

clean_alphabetically

,Title,publisher,publishedDate,authors,categories,ratings_average,ratings_std,ratings_count
118557,""" Film technique, "" and, "" Film acting """,Sims Press,2008-11,[V. I. Pudovkin],Drama,4.500000,0.707107,2
28608,""" We'll Always Have Paris"": The Definitive Gui...",Perennial,1994,"[Robert A. Nowlan, Gwendolyn Wright Nowlan]",Reference,5.000000,0.000000,2
113956,"""... And Poetry is Born ..."" Russian Classical...",None,1984,[Aleksandr Sergeevich Pushkin],Russian poetry,4.000000,NaN,1
209946,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,1913,[Shan F. Bullock],None,4.875000,0.353553,8
196186,"""A Truthful Impression of the Country"": Britis...",University of Michigan Press,2001,"[Nicholas J. Clifford, Nicholas Rowland Cliffo...",History,4.000000,NaN,1
...,...,...,...,...,...,...,...,...
91449,with an everlasting love,Harvest House Publishers,1999-07-01,[Kay Arthur],Religion,4.761905,0.889087,21
64773,work and Motivation,SAGE,2012,[Gary P. Latham],Business & Economics,5.000000,NaN,1
84075,www.whitbread.org/book,None,None,[],None,2.666667,2.081666,3
153555,xBase Programming for the True Beginner: An In...,McGraw-Hill/Irwin,1995-11-01,"[Eugene Kaluzniacky, Vijay Kanabar]",Computers,5.000000,NaN,1


In [20]:
threshold = clean_alphabetically[clean_alphabetically["ratings_count"] > 100]
print(len(clean_alphabetically[clean_alphabetically["ratings_count"] > 100]))
threshold["ratings_count"].sum()

3983


1499646

In [21]:
# Group reviews by title and get a list of indices for each title
indices_per_title = reviews.groupby('Title').apply(lambda x: x.index.tolist())

# Convert the result to a Pandas Series
indices_series = pd.Series(indices_per_title, name='review_ids').sort_index()

C:\Users\LucasSchepers\AppData\Local\Temp\ipykernel_19224\1176013281.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indices_per_title = reviews.groupby('Title').apply(lambda x: x.index.tolist())


In [22]:
indices_series

Title
" Film technique, " and, " Film acting "                                                                                                                                  [1664106, 1664107]
" We'll Always Have Paris": The Definitive Guide to Great Lines from the Movies                                                                                             [402728, 402729]
"... And Poetry is Born ..." Russian Classical Poetry                                                                                                                              [1604731]
"A Titanic hero" Thomas Andrews, shipbuilder                                                                                               [2783512, 2783513, 2783514, 2783515, 2783516, ...
"A Truthful Impression of the Country": British and American Travel Writing in China, 1880-1949                                                                                    [2588738]
                                                 

In [23]:
# Verifying it works
titanic_indices = indices_series.iloc[3]
print(titanic_indices, type(titanic_indices[0]))
reviews.loc[titanic_indices]

[2783512, 2783513, 2783514, 2783515, 2783516, 2783517, 2783518, 2783519] <class 'int'>


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
2783512,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,A46JFD3YDSEB6,only me,6/6,5.0,914716800,A man who shone like a star,Anyone who has ever looked at Thomas Andrews' ...
2783513,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,A1KW23XZW6LTAB,"Eileen Grimes ""Titanic Astrology author""",4/4,5.0,969840000,God bless this man,This book was such a lovely find; I had been f...
2783514,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,A11QO67YPZ05CX,"""darl85""",4/4,5.0,916444800,A man loved and respected by so many,Shan F.Bullock truely captures the essence of ...
2783515,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,A305WWFYE89S6O,thomas e. lewis,2/2,4.0,947894400,A must read for every Titanic Buff!,I took a tour of a local titanic display not t...
2783516,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,None,None,2/2,5.0,908236800,A wonderful book for such a wonderful man.,When I first saw James Cameron's movie Titanic...
2783517,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,ADMPGBI8ACXDN,first.officer@cfu-cybernet.net,1/1,5.0,903830400,"A dry, yet riveting tale of a man destined for...",The stories of Mr. Andrews' life were captivat...
2783518,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,A3ADZU0KM5KNW6,Joel Grissom,0/0,5.0,946944000,the best true book ever!,Andrews is a man who died with his ship and sh...
2783519,B00087XUJ8,"""A Titanic hero"" Thomas Andrews, shipbuilder",None,AMQU5P20MYAC2,ntlelmbrt@yahoo.com,0/0,5.0,900892800,An absolutely charming chronicle of Thomas And...,Keeping in mind the era that this book was wri...


In [24]:
clean_alphabetically["review_ids"] = indices_series.tolist()

In [25]:
# sort it by index again
clean_books = clean_alphabetically.sort_index()
clean_books

,Title,publisher,publishedDate,authors,categories,ratings_average,ratings_std,ratings_count,review_ids
0,Its Only Art If Its Well Hung!,None,1996,[Julie Strain],Comics & Graphic Novels,4.000000,NaN,1,[0]
1,Dr. Seuss: American Icon,A&C Black,2005-01-01,[Philip Nel],Biography & Autobiography,4.555556,0.527046,9,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
2,Wonderful Worship in Smaller Churches,None,2000,[David R. Ray],Religion,5.000000,0.000000,4,"[10, 11, 12, 13]"
3,Whispers of the Wicked Saints,iUniverse,2005-02,[Veronica Haddon],Fiction,3.718750,1.764056,32,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2..."
4,"Nation Dance: Religion, Identity and Cultural ...",None,2003-03-01,[Edward Long],None,5.000000,NaN,1,[46]
...,...,...,...,...,...,...,...,...,...
212399,The Orphan Of Ellis Island (Time Travel Advent...,Scholastic Paperbacks,2000-06-01,[Elvira Woodruff],Juvenile Fiction,4.678571,0.547964,28,"[2821541, 2821542, 2821543, 2821544, 2821545, ..."
212400,Red Boots for Christmas,None,1995,[],Juvenile Fiction,5.000000,0.000000,2,"[2821569, 2821570]"
212401,Mamaw,None,2018-01-17,[Wild Wild Cabbage],None,4.666667,0.577350,3,"[2821571, 2821572, 2821573]"
212402,The Autograph Man,Vintage,2003-08-12,[Zadie Smith],Fiction,2.500000,1.290994,4,"[2821574, 2821575, 2821576, 2821577]"


In [26]:
# This is to show the length of the review_id's list is actually the same as the rating_count
lens = clean_books["review_ids"].apply(len)
bools = lens == clean_books["ratings_count"]
bools.prod()

1

## Separate the review data in true review data and user data

In [27]:
print("Number of unique User_id's", reviews["User_id"].nunique())
print("Number of unique profileNames: ", reviews["profileName"].nunique())

Number of unique User_id's 1008961
Number of unique profileNames:  854140


In [28]:
result = reviews.groupby('User_id')['profileName'].nunique() == 1

count = 0
for key, value in result.items():
    if not value:
        count += 1
print("Number of user_id's with multiple profileName: ", count)

Number of user_id's with multiple profileName:  461


In [29]:
result = reviews.groupby('profileName')['User_id'].nunique() == 1

count = 0
for key, value in result.items():
    if not value:
        count += 1
        print(key)
print("Number of profileNames with multiple User_id: ", count)





!!!!!!!!
"unknown"
#1
#21
*
-
--
-.-
.
. "."
...
000
007
1
10
1125
123
1234
12345
123456
123456789
16 year old
1L
1st grade teacher
1st time Mom
1st time mom
2bluesky2
2cents
2much2do
35
3L
4eyes
7th Grade Student
:(
:)
?
A
A "A"
A ...reader
A 10-year old reader
A 12-year old gamer
A 12-year old reader
A 12-year old viewer
A 14 year old girl
A 14 year old reader
A 14-year-old reader
A 5-year old reader
A 7-year old reader
A 9-year old reader
A 9-year old toy enthusiast
A Book A Day
A Book Lover
A Brooks
A Canadian Reader
A Christian
A Christian reader
A Constant Reader
A Consumer
A Critical Reader
A Customer
A Customer "A Customer"
A Disappointed Reader
A Discerning Reader
A Fan
A Friend
A G "AG"
A Girl
A Grad Student
A Graduate Student
A H
A Japanese
A Jones
A Kindergarten Teacher
A Kutztown University Student
A Lady
A Listener
A Long Time Fan
A Marine
A Mom
A Mommy
A Mother
A Music Fan
A Music Lover
A New Yorker
A Parent
A Pastor
A Pen Name
A Person
A Physician
A

In [30]:
reviews[reviews["profileName"]=="A 10-year old reader"]

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
65142,B0007K33F2,"Monte Cristo,",None,A2F5PZPU3B2E27,A 10-year old reader,0/0,5.0,999734400,"Truly, A Masterpiece",The Count of Monte Cristo is touching story ab...
79185,0606035826,Garden of Shadows (Dollanganger Series),None,AFXT6PPW1ZTQA,A 10-year old reader,7/14,1.0,1007769600,This story line is getting really really reall...,I loved FLOWERS IN THE ATTIC. PETALS ON THE WI...
88237,B000NWU3I4,"The Hobbitt, or there and back again; illustra...",None,A1H4K5YVQ76MVW,A 10-year old reader,0/0,5.0,999388800,A wild adventure with some wild friends,This book takes you on a magical adventure and...
98357,B0001BJEF0,So You Want to Be a Wizard: Young Wizard Serie...,None,A35TGFHPGFYLI4,A 10-year old reader,1/2,4.0,1015027200,So You Want to Be a Wizard,I thought that this book was a very good book....
153357,0440228840,Demon in My View (Den of Shadows),6.99,A31Q3B4HVHA5ZY,A 10-year old reader,0/0,3.0,999561600,Demon in My View,"I love this book, the one person I like was, A..."
...,...,...,...,...,...,...,...,...,...,...
2896442,B000MW8NVY,CHARLIE AND THE GREAT GLASS ELEVATOR,None,A1ZN0YS0POPHE8,A 10-year old reader,3/5,5.0,1008806400,Visit Willy Wonka&#65533;s Wondrous World Again!,Charlie and the Great Glass ElevatorVisit Will...
2911382,0856921645,Christmas Carols for Young Children: For Piano...,None,A14UM9S0CH7751,A 10-year old reader,3/4,5.0,1008115200,The Most Beautifully Illustrated Music Book Ever,This music book has the most beautiful illustr...
2942725,B000H9R1Q0,The Hobbit,None,A1H4K5YVQ76MVW,A 10-year old reader,0/0,5.0,999388800,A wild adventure with some wild friends,This book takes you on a magical adventure and...
2973583,B000MTRTTE,The Count of Monte Cristo,None,A2F5PZPU3B2E27,A 10-year old reader,0/0,5.0,999734400,"Truly, A Masterpiece",The Count of Monte Cristo is touching story ab...


In [31]:
missing_id_and_name = reviews[(reviews['User_id'].isnull()) * (reviews['profileName'].isnull())]
len(missing_id_and_name)

561774

In [32]:
reviews.count()

Id                    2999792
Title                 2999792
Price                  481164
User_id               2438018
profileName           2437900
review/helpfulness    2999792
review/score          2999792
review/time           2999792
review/summary        2999385
review/text           2999784
dtype: int64

In [33]:
missing_id_and_name = reviews[reviews['User_id'].isnull()][reviews['profileName'].isnull()]
len(missing_id_and_name)

C:\Users\LucasSchepers\AppData\Local\Temp\ipykernel_19224\3072194807.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missing_id_and_name = reviews[reviews['User_id'].isnull()][reviews['profileName'].isnull()]


561774

HYPOTHESIS: The profile name is the screen name appearing. Many submission may have come with the same User_ID. Multiple Profile Names per user_ID are due to accounts changing names. Let's not look at the "ProfileName" category too much.

TO DO: 
- make an overview per UserID with their associated profile names (may be multiple).
- Include in this overview the average rating that user_id gives and how many books they have rated.
- Create a trimmed down reviews table countaining only Titles, User_id's and scores. 

In [34]:
tab = reviews.groupby('User_id')["Title"].nunique()
tab[tab>100]

User_id
A106016KSI0YQ     205
A10T0OW97SFBB     182
A10VOEBL5S337W    108
A114YQ7ZT9Y1W5    214
A11B61QBGHLQDN    163
                 ... 
AXW1O6Q92XF0Z     139
AYCJSA9HR7TKO     116
AYFNOHE3IQQ4G     150
AYPCUQS6ARWFH     108
AYT4FJYVCHYLE     200
Name: Title, Length: 746, dtype: int64

In [35]:
tab2 = reviews.groupby('Title').nunique()

In [36]:
reviews[reviews["User_id"] == "AZZZT14MS21I6"]

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
1375658,0963869531,One More Bridge to Cross: Lowering the Cost of...,9.5,AZZZT14MS21I6,Johan Strm,17/17,5.0,973987200,Mandatory reading!,One More Bridge to Cross is a must read for an...
1381976,0963869558,Phantom Soldier: The Enemy's Answer to U.S. Fi...,10.21,AZZZT14MS21I6,Johan Strm,12/12,5.0,1044057600,If you want to win - read this book!,Phantom Soldier is the natural follow-up to Jo...
1382870,0963869566,The Tiger's Way: A U.S. Private's Best Chance ...,11.66,AZZZT14MS21I6,Johan Strm,13/15,5.0,1094688000,Future doctrine in making,This is John Poole's best writing yet.The Tige...
1385023,0963869574,Tactics of the Crescent Moon: Militant Muslim ...,10.91,AZZZT14MS21I6,Johan Strm,6/8,5.0,1105488000,Another life-saver by John Poole,Tactics of the Crescent Moon is a survival gui...
1388155,0963869582,Militant Tricks: Battlefield Ruses of the Isla...,10.26,AZZZT14MS21I6,Johan Strm,6/6,5.0,1139875200,Get up to speed - quick,This book is a must read.The clarity with whic...


## Export files

In [41]:
start_time = time.time()
reviews.to_csv('data_sets/reviews_clean.csv', index=False)
end_time = time.time()
print("Import time:", end_time - start_time, "seconds")

Import time: 67.74606561660767 seconds


In [42]:
reviews_small = reviews.copy()

del reviews_small["Price"]
del reviews_small["review/text"]
del reviews_small["review/summary"]
del reviews_small["review/time"]
del reviews_small["review/helpfulness"]
del reviews_small["Id"]
del reviews_small["profileName"]

reviews_small.to_csv('data_sets/reviews_for_recommenders.csv', index=False)

In [43]:
reviews_small

,Title,User_id,review/score
0,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0
2,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0
3,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0
...,...,...,...
2999995,The Idea of History,None,4.0
2999996,The Idea of History,A1SMUB9ASL5L9Y,4.0
2999997,The Idea of History,A2AQMEKZKK5EE4,4.0
2999998,The Idea of History,A18SQGYBKS852K,5.0


In [44]:
clean_books.to_csv('data_sets/books_clean.csv', index=False)
clean_books

,Title,publisher,publishedDate,authors,categories,ratings_average,ratings_std,ratings_count,review_ids
0,Its Only Art If Its Well Hung!,None,1996,[Julie Strain],Comics & Graphic Novels,4.000000,NaN,1,[0]
1,Dr. Seuss: American Icon,A&C Black,2005-01-01,[Philip Nel],Biography & Autobiography,4.555556,0.527046,9,"[1, 2, 3, 4, 5, 6, 7, 8, 9]"
2,Wonderful Worship in Smaller Churches,None,2000,[David R. Ray],Religion,5.000000,0.000000,4,"[10, 11, 12, 13]"
3,Whispers of the Wicked Saints,iUniverse,2005-02,[Veronica Haddon],Fiction,3.718750,1.764056,32,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2..."
4,"Nation Dance: Religion, Identity and Cultural ...",None,2003-03-01,[Edward Long],None,5.000000,NaN,1,[46]
...,...,...,...,...,...,...,...,...,...
212399,The Orphan Of Ellis Island (Time Travel Advent...,Scholastic Paperbacks,2000-06-01,[Elvira Woodruff],Juvenile Fiction,4.678571,0.547964,28,"[2821541, 2821542, 2821543, 2821544, 2821545, ..."
212400,Red Boots for Christmas,None,1995,[],Juvenile Fiction,5.000000,0.000000,2,"[2821569, 2821570]"
212401,Mamaw,None,2018-01-17,[Wild Wild Cabbage],None,4.666667,0.577350,3,"[2821571, 2821572, 2821573]"
212402,The Autograph Man,Vintage,2003-08-12,[Zadie Smith],Fiction,2.500000,1.290994,4,"[2821574, 2821575, 2821576, 2821577]"
